<a href="https://colab.research.google.com/github/VincentChen2/Emotion-Detector-NAISS-Spring-2024/blob/main/Emotion_Detector_NAISS_Spring_2024_Test_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Dataset


In [ ]:
num_of_folders = int(input('Input Number of Folders: '))
folder_names = []

for n in range(num_of_folders):
    try:
        folder_name = f"{input('Input Folder Name: ')}"
        !unzip NeutralExpressions #TEMP FIX: Change this line for each zip file until fix is found
        folder_names.append(folder_name)
    except:
        print('Zip File Cannot be Found')

# Model

In [7]:
#Imports
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split

In [17]:
print(folder_names)
Categories = []
for folder_name in folder_names:
    num_of_files = len(os.listdir(folder_name))
    cat_id = 0

    for i in range(num_of_files):
        im = cv2.imread(f'{folder_name}/IMG_{i}.jpg', cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, dsize = (100, 100))
        if not (im[0][0] == 0 and im[0][1] == 0):
            Categories.append((im, cat_id))

    cat_id += 1

['NeutralExpressions', 'HappyExpressions']


In [ ]:
plt.imshow(Categories[60][0])

In [23]:
totalImages = Categories # +Other Image Arrays

In [ ]:
plt.imshow(totalImages[53][0])

In [26]:
X = [x[0] for x in totalImages]
X = np.array(X)
X = np.reshape(X, (len(totalImages), 100, 100, 1))
Y = [x[1] for x in totalImages]
Y = np.array(Y)
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.2)

In [28]:
x1 = tf.keras.layers.Input((100, 100, 1))
x2 = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu')(x1)
x3 = tf.keras.layers.MaxPooling2D((2, 2))(x2)
x4 = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu')(x3)
x5 = tf.keras.layers.MaxPooling2D((2, 2))(x4)
x6 = tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu')(x5)
x7 = tf.keras.layers.MaxPooling2D((2, 2))(x6)
f = tf.keras.layers.Flatten()(x7)
d1 = tf.keras.layers.Dense(32, activation = 'relu')(f)
d2 = tf.keras.layers.Dense(16, activation = 'relu')(f)
d3 = tf.keras.layers.Dense(1, activation = 'sigmoid')(f)

model = tf.keras.models.Model(inputs = x1, outputs = d3)

In [29]:
model.summary()
if num_of_folders < 2:
    print('Insufficient Number of Categories; Need at Least Two Categories')
elif num_of_folders == 2:
    model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])
else:
    model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 98, 98, 64)        640       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 64)        36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 21, 21, 64)        3692

In [31]:
model.fit(xtrain, ytrain, batch_size = 10, epochs = 100)

Epoch 1/100
24/24 [==============================] - 7s 302ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/100
24/24 [==============================] - 6s 247ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/100
24/24 [==============================] - 7s 303ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/100
24/24 [==============================] - 6s 244ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/100
24/24 [==============================] - 8s 355ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/100
24/24 [==============================] - 7s 280ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/100
24/24 [==============================] - 8s 315ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/100
24/24 [==============================] - 7s 300ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/100
17/24 [====================>.........] - ETA: 1s - loss: 0.0000e+00 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
totalCorrect = 0
predictions = model.predict(xtest)
for i in range(len(predictions)):
  binary = np.round_(predictions[i])
  if binary == ytest[i]:
    totalCorrect += 1
print(totalCorrect / len(ytest))

#Saving to Drive from Colab

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
model_path = "/content/drive/My Drive/EmotionModel.h5"
model.save(model_path)